Example MLP from the *Flux Quick Start documentation*.

Hopefully we've already got everything we need installed into our
project package and a suitable *Jupyter* kernel to use this environment.
If not see [how to set up environment](../INSTALL.org).

The real source of this *HelloANNWorld* is here and I merely reproduce
it in org source form as a test of the setup:

\[\[<https://fluxml.ai/Flux.jl/stable/models/quickstart/#man-quickstart>\]\]

# A Neural Network in One Minute

If you have used neural networks before, then this simple example might
be helpful for seeing how the major parts of *Flux* work together.

In [ ]:

using Flux, Metal, Statistics, ProgressMeter

# Data  for XOR problem
# 2x1000 random Float32 matrix
noisy = rand(Float32, 2, 1000)
# 1000 element vector of Bool
truth = [xor(col[1]>0.5, col[2]>0.5) for col in eachcol(noisy)]

# MLP with one hidden layer of size 3
model = Chain(
    Dense(2 => 3, tanh),
    BatchNorm(3),
    Dense(3 => 2),
    softmax) |> gpu

# The model encapsulates parameters, randomly initialised. Its initial output is:
out1 = model(noisy |> gpu) |> cpu # 2×1000 Matrix{Float32}

# To train the model, we use batches of 64 samples, and one-hot encoding:
target = Flux.onehotbatch(truth, [true, false]) # 2×1000 OneHotMatrix
loader = Flux.DataLoader((noisy, target) |> gpu, batchsize=64, shuffle=true);
# 16-element DataLoader with first element: (2×64 Matrix{Float32}, 2×64 OneHotMatrix)

optim = Flux.setup(Flux.Adam(0.01), model)  # will store optimiser momentum, etc.

# Training loop, using the whole data set 1000 times:
losses = []
@showprogress for epoch in 1:1_000
    for (x, y) in loader
        loss, grads = Flux.withgradient(model) do m
            # Evaluate model and loss inside gradient context:
            y_hat = m(x)
            Flux.crossentropy(y_hat, y)
        end
        Flux.update!(optim, model, grads[1])
        push!(losses, loss)  # logging, outside gradient context
    end
end

optim # parameters, momenta and output have all changed
out2 = model(noisy |> gpu) |> cpu  # first row is prob. of true, second row p(false)

mean((out2[1,:] .> 0.5) .== truth)  # accuracy 94% so far!


In [ ]:
using Plots

p_true = scatter(noisy[1,:], noisy[2,:], zcolor=truth, title="True classification", legend=false)
p_raw =  scatter(noisy[1,:], noisy[2,:], zcolor=out1[1,:], title="Untrained network", label="", clims=(0,1))
p_done = scatter(noisy[1,:], noisy[2,:], zcolor=out2[1,:], title="Trained network", legend=false)

plot(p_true, p_raw, p_done, layout=(1,3), size=(750,250))


## Loss during training

In [ ]:
plot(losses; xaxis=(:log10, "iteration"), yaxis="loss", label="per batch")
n = length(loader)
plot!(n:n:length(losses), mean.(Iterators.partition(losses, n)), label="epoch mean", dpi=200)


This XOR ("exclusive or") problem is a variant of the famous one which
drove Minsky and Papert to invent deep neural networks in 1969. For
small values of "deep" – this has one hidden layer, while earlier
perceptrons had none. (What they call a hidden layer, Flux calls the
output of the first layer, model\[1\](noisy).)